In [ ]:
import numpy as np
from scipy.signal import convolve2d
import tensorflow as tf
import keras
from keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from random import randint, random
import sys
import gc

In [ ]:
def life_step(X):
    """Game of life step using scipy tools"""
    X_pad = np.zeros((X.shape[0], X.shape[1]+2, X.shape[2]+2))
    N = np.zeros((X.shape[0], X.shape[1]+2, X.shape[2]+2))
    X_pad[:,1:-1,1:-1] += X
    
    X_pad[:, 0, 1:-1] = X[:, -1, :]
    X_pad[:, -1, 1:-1] = X[:, 0, :]
    
    X_pad[:, 1:-1, 0] = X[:, :, -1]
    X_pad[:, 1:-1, -1] = X[:, :, 0]
    
    X_pad[:, 0, 0] = X[:, -1, -1]
    X_pad[:, 0, -1] = X[:, -1, 0]
    X_pad[:, -1, 0] = X[:, 0, -1]
    X_pad[:, -1, -1] = X[:, 0, 0]
    
    N[:, 1:, 1:] += X_pad[:,:-1,:-1]
    N[:, 1:, :] += X_pad[:,:-1,:]
    N[:, 1:, :-1] += X_pad[:,:-1,1:]

    N[:, :, 1:] += X_pad[:,:,:-1]
    N[:, :, :-1] += X_pad[:,:,1:]

    N[:, :-1, 1:] += X_pad[:,1:,:-1]
    N[:, :-1, :] += X_pad[:,1:,:]
    N[:, :-1, :-1] += X_pad[:,1:,1:]
    
    N = N[:,1:-1,1:-1]

    return np.logical_or(N == 3, np.logical_and(X, N==2)).astype(np.uint8)


In [ ]:
train = pd.read_csv("../input/conways-reverse-game-of-life-2020/train.csv", index_col='id').values
test = pd.read_csv("../input/conways-reverse-game-of-life-2020/test.csv", index_col='id').values

train_delta = train[:,0]
test_delta = test[:,0]

X_train = train[:, 1:626].reshape((-1, 25, 25))
y_train = train[:, 626:].reshape((-1, 25, 25))
X_test = test[:, 1:626].reshape((-1, 25, 25))

print(train_delta.shape, X_train.shape, y_train.shape)
print(test_delta.shape, X_test.shape)

In [ ]:
current_step = X_train
stop = []
start = []
for i in range(5):
    current_mask = train_delta >= i+1
    start.append(current_step[current_mask])
    current_step = life_step(current_step)
    stop.append(current_step[current_mask])
X_train = np.concatenate(stop, axis=0)
y_train = np.concatenate(start, axis=0)
print(X_train.shape, y_train.shape, train_delta.sum())

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33)

In [ ]:
p0 = 0.3058699275697248
def iterate_training_batch(batch_size):
    ids = np.random.permutation(np.arange(len(X_train)))
    while 1:
        #y = np.random.randint(0, 2, (batch_size, game_dim, game_dim))
        #X = life_step(y)
        
        #yield (np.expand_dims(X.astype(np.float32), axis=-1), np.expand_dims(y.astype(np.float32), axis=-1))
        batch_mask = ids[:batch_size]
        ids = ids[batch_size:]
        
        if len(ids) < batch_size:
            ids = np.random.permutation(np.arange(len(X_train)))
            
        X = X_train[batch_mask]
            
        X_pad = np.zeros((X.shape[0], X.shape[1]+2, X.shape[2]+2))
        N = np.zeros((X.shape[0], X.shape[1]+2, X.shape[2]+2))
        X_pad[:,1:-1,1:-1] += X

        X_pad[:, 0, 1:-1] = X[:, -1, :]
        X_pad[:, -1, 1:-1] = X[:, 0, :]

        X_pad[:, 1:-1, 0] = X[:, :, -1]
        X_pad[:, 1:-1, -1] = X[:, :, 0]

        X_pad[:, 0, 0] = X[:, -1, -1]
        X_pad[:, 0, -1] = X[:, -1, 0]
        X_pad[:, -1, 0] = X[:, 0, -1]
        X_pad[:, -1, -1] = X[:, 0, 0]

        N[:, 1:, 1:] += X_pad[:,:-1,:-1]
        N[:, 1:, :] += X_pad[:,:-1,:]
        N[:, 1:, :-1] += X_pad[:,:-1,1:]

        N[:, :, 1:] += X_pad[:,:,:-1]
        N[:, :, :-1] += X_pad[:,:,1:]

        N[:, :-1, 1:] += X_pad[:,1:,:-1]
        N[:, :-1, :] += X_pad[:,1:,:]
        N[:, :-1, :-1] += X_pad[:,1:,1:]

        N = N[:,1:-1,1:-1]
        
        X_indic = np.ones_like(X) * p0
        indic_mask = np.logical_and(N==0, X==0)
        X_indic[indic_mask] = 0
        prop_clue = np.repeat(np.random.random(batch_size), X.shape[-2] * X.shape[-1]).reshape(X.shape)
        clues_mask = np.random.random(X.shape) < prop_clue
        X_indic[clues_mask] = y_train[batch_mask][clues_mask]

        X = np.concatenate((np.expand_dims(X, axis=-1), np.expand_dims(X_indic, axis=-1)), axis=-1)

        yield X.astype(np.float32), np.expand_dims(y_train[batch_mask].astype(np.float32), axis=-1)

def iterate_val_batch(batch_size):
    ids = np.random.permutation(np.arange(len(X_val)))
    while 1:
        #y = np.random.randint(0, 2, (batch_size, game_dim, game_dim))
        #X = life_step(y)
        
        #yield (np.expand_dims(X.astype(np.float32), axis=-1), np.expand_dims(y.astype(np.float32), axis=-1))
        batch_mask = ids[:batch_size]
        ids = ids[batch_size:]
        
        if len(ids) < batch_size:
            ids = np.random.permutation(np.arange(len(X_val)))
            
        X = X_val[batch_mask]

        X_pad = np.zeros((X.shape[0], X.shape[1]+2, X.shape[2]+2))
        N = np.zeros((X.shape[0], X.shape[1]+2, X.shape[2]+2))
        X_pad[:,1:-1,1:-1] += X

        X_pad[:, 0, 1:-1] = X[:, -1, :]
        X_pad[:, -1, 1:-1] = X[:, 0, :]

        X_pad[:, 1:-1, 0] = X[:, :, -1]
        X_pad[:, 1:-1, -1] = X[:, :, 0]

        X_pad[:, 0, 0] = X[:, -1, -1]
        X_pad[:, 0, -1] = X[:, -1, 0]
        X_pad[:, -1, 0] = X[:, 0, -1]
        X_pad[:, -1, -1] = X[:, 0, 0]

        N[:, 1:, 1:] += X_pad[:,:-1,:-1]
        N[:, 1:, :] += X_pad[:,:-1,:]
        N[:, 1:, :-1] += X_pad[:,:-1,1:]

        N[:, :, 1:] += X_pad[:,:,:-1]
        N[:, :, :-1] += X_pad[:,:,1:]

        N[:, :-1, 1:] += X_pad[:,1:,:-1]
        N[:, :-1, :] += X_pad[:,1:,:]
        N[:, :-1, :-1] += X_pad[:,1:,1:]

        N = N[:,1:-1,1:-1]
        
        X_indic = np.ones_like(X) * p0
        indic_mask = np.logical_and(N==0, X==0)
        X_indic[indic_mask] = 0
        prop_clue = np.repeat(np.random.random(batch_size), X.shape[-2] * X.shape[-1]).reshape(X.shape)
        clues_mask = np.random.random(X.shape) < prop_clue
        X_indic[clues_mask] = y_val[batch_mask][clues_mask]
        
        X = np.concatenate((np.expand_dims(X, axis=-1), np.expand_dims(X_indic, axis=-1)), axis=-1)
                        
        yield X.astype(np.float32), np.expand_dims(y_val[batch_mask].astype(np.float32), axis=-1)
         

In [ ]:
game_dim = 25
batch_size = 128

INITIAL_CONV = 64
RES_LAYERS = 6

In [ ]:
class ResnetIdentityBlock(keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

def get_model(input_shape):
  t_input = keras.layers.Input(input_shape)
  x = keras.layers.Conv2D(INITIAL_CONV, 3, padding='same')(t_input)
  x = keras.layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  for _ in range(RES_LAYERS):
    x_input = x
    x = keras.layers.Conv2D(INITIAL_CONV, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = tf.nn.relu(x)

    x = keras.layers.Conv2D(INITIAL_CONV, 3, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)

    x += x_input
    x = keras.layers.Conv2D(1, 3, padding='same')(x)
    x = keras.activations.sigmoid(x)

  return keras.models.Model(inputs=t_input, outputs=x)

def build_model():
  return get_model((game_dim, game_dim, 2))

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
latent_dim = 20

def get_encoder():
    encoder_inputs = keras.Input(shape=(game_dim, game_dim, 1))
    x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
    
    x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(16, activation="relu")(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
    encoder.summary()
    return encoder

def get_decoder():
    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.Dense(6 * 6 * 64, activation="relu")(latent_inputs)
    x = layers.Reshape((6, 6, 64))(x)
    x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
    
    x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="valid")(x)
    
    x = keras.activations.sigmoid(x)
    decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
    decoder.summary()
    return decoder

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        
    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        return self.decoder(z)

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 25 * 25
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

def get_VAE():
    return VAE(get_encoder(), get_decoder())

In [ ]:
model = build_model()

model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001, verbose=1)

file_path = 'best_model.hdf5'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='val_loss',
                                                    save_best_only=True)

callbacks = [reduce_lr, model_checkpoint]
#callbacks = [model_checkpoint]

model.summary()

In [ ]:
hist = model.fit(
    iterate_training_batch(batch_size),
    steps_per_epoch=len(X_train)//batch_size,
    epochs=25,
    verbose=1,
    callbacks=callbacks,
    validation_data=iterate_val_batch(batch_size),
    validation_steps=len(X_val)//batch_size,
    shuffle=True
)

In [ ]:
del X_train
del y_train
del X_val
del y_val
del train
del train_delta
gc.collect()

In [ ]:
model.load_weights('./best_model.hdf5')

In [ ]:
def get_padded_version_n(X):
    X_pad = np.zeros((X.shape[0], X.shape[-2] + 2, X.shape[-1] + 2), dtype=X.dtype)
    X_pad[:, 1:-1,1:-1] += X
    
    X_pad[:, 0, 1:-1] = X[:, -1, :]
    X_pad[:, -1, 1:-1] = X[:, 0, :]
    
    X_pad[:, 1:-1, 0] = X[:, :, -1]
    X_pad[:, 1:-1, -1] = X[:, :, 0]
    
    X_pad[:, 0, 0] = X[:, -1, -1]
    X_pad[:, 0, -1] = X[:, -1, 0]
    X_pad[:, -1, 0] = X[:, 0, -1]
    X_pad[:, -1, -1] = X[:, 0, 0]
    
    return X_pad

def solve(F, d):
    sF = F
    P = p0 * np.ones_like(F)
    
    F_pad = get_padded_version_n(F)
    
    N = np.zeros(F_pad.shape)
    
    N[:, 1:, 1:] += F_pad[:,:-1,:-1]
    N[:, 1:, :] += F_pad[:,:-1,:]
    N[:, 1:, :-1] += F_pad[:,:-1,1:]

    N[:, :, 1:] += F_pad[:,:,:-1]
    N[:, :, :-1] += F_pad[:,:,1:]

    N[:, :-1, 1:] += F_pad[:,1:,:-1]
    N[:, :-1, :] += F_pad[:,1:,:]
    N[:, :-1, :-1] += F_pad[:,1:,1:]

    N = N[:,1:-1,1:-1]
    
    P[np.logical_and(N==0, F==0)] = 0
    print("Starting from", np.logical_and(N==0, F==0).sum(axis=(-1, -2)) / (F.shape[-2] * F.shape[-1]))
    
    I = np.zeros_like(F)
    global_mask = np.ones(F.shape[0]).astype(np.bool_)
    
    while len(F):
        X = np.concatenate((np.expand_dims(F, axis=-1), np.expand_dims(P, axis=-1)), axis=-1)
        y = model.predict(X)[:, :, :, 0]
        del X
        y[P==0.0] = 0
        y[P==1.0] = 1
        H = abs(y - 0.5)
        H_eff = (H * (H<0.5).astype(np.float32))
        change_mask = H == np.repeat(H_eff.max(axis=(-1, -2)), F.shape[-2] * F.shape[-1]).reshape(H.shape)
        nP = y
        p_step = nP[change_mask]
        nP[change_mask] = np.round(p_step)
        
        qsure = ((nP==0.0).sum(axis=(-1, -2)) + (nP==1.0).sum(axis=(-1, -2))) / (F.shape[-2] * F.shape[-1])
        local_continue_mask = qsure < 1.0
        local_stop_mask = np.logical_not(local_continue_mask)
        
        if local_stop_mask.sum() > 0:
            eq = (life_step(nP[local_stop_mask]) == F[local_stop_mask])
            I[np.arange(len(I))[global_mask][local_stop_mask]] = nP[local_stop_mask]
            d[np.arange(len(d))[global_mask][local_stop_mask]] -= 1
            F[local_stop_mask] = nP[local_stop_mask]
            
            nF = F[local_stop_mask]
            n_p = p0 * np.ones_like(nF)
            nF_pad = get_padded_version_n(nF)

            nN = np.zeros_like(nF_pad)

            nN[:, 1:, 1:] += nF_pad[:,:-1,:-1]
            nN[:, 1:, :] += nF_pad[:,:-1,:]
            nN[:, 1:, :-1] += nF_pad[:,:-1,1:]

            nN[:, :, 1:] += nF_pad[:,:,:-1]
            nN[:, :, :-1] += nF_pad[:,:,1:]

            nN[:, :-1, 1:] += nF_pad[:,1:,:-1]
            nN[:, :-1, :] += nF_pad[:,1:,:]
            nN[:, :-1, :-1] += nF_pad[:,1:,1:]

            nN = nN[:,1:-1,1:-1]
            
            n_p[np.logical_and(nN==0, nF==0)] = 0
            
            nP[local_stop_mask] = n_p
            
            local_continue_mask[local_stop_mask] = d[global_mask][local_stop_mask] > 0
            F = F[local_continue_mask]
            nP = nP[local_continue_mask]
            """
            print("Done for {}, steps {}, continuing with {} items".format(
                np.arange(len(I))[global_mask][local_stop_mask],
                d[global_mask][local_stop_mask],
                local_continue_mask.sum()
            ))
            """
            print(local_continue_mask.sum(), " items left")
            global_mask[global_mask] = local_continue_mask
            
            del eq
            del nN
            del nF
            del nF_pad
        P = nP
        del y
        del H
        del H_eff
        del change_mask
        del p_step
        del qsure
        del local_continue_mask
        del local_stop_mask
        gc.collect()
    return I

In [ ]:
def get_optimized_solution_and_score(y, delta, labels):
    print("Scores from prediction")
    res = np.zeros_like(y)
    scores = []
    current_step = np.copy(y)
    for i in range(5):
        current_step = life_step(current_step)
        scores.append(current_step[delta==i+1] == labels[delta==i+1])
        print("Actual score for delta={}: {}".format(i+1, scores[-1].mean()))
        res[delta == i+1] = current_step[delta == i+1]
    print("Actual LB score = ", 1 - (res == labels).mean())
    y_scores = (labels == res).mean(axis=(-2, -1))
    blank_scores = (labels==0).mean(axis=(-2, -1))
    score_mask = blank_scores > y_scores
    y_final = np.copy(y)
    y_final[score_mask] = 0
    res[score_mask] = 0
    print("Scores from final response")
    scores = []
    for i in range(5):
        scores.append(res[delta==i+1] == labels[delta==i+1])
        print("Score for delta={}: {}".format(i+1, scores[-1].mean()))
    f_score = 1 - (res == labels).mean()
    print("LB score estimation = ", f_score)
    return y_final, f_score

In [ ]:
y = solve(X_test, np.copy(test_delta))

In [ ]:
y_final, score = get_optimized_solution_and_score(y, test_delta, X_test)
blank_score = 1 - (X_test == 0).mean()
print(score, blank_score - score)

In [ ]:
submission = pd.read_csv("../input/conways-reverse-game-of-life-2020/sample_submission.csv", index_col='id')
submission_values = y_final.reshape((len(submission), 625))
for i, col in enumerate(submission.columns):
    submission[col].values[:] = submission_values[:, i]


In [ ]:
submission.to_csv("submission.csv")